In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls 'gdrive/My Drive/'

'2019-04-11 18-17.pdf'			   ESN-growth-test.ipynb
 8-8-ESN-G.hdf5				   ESN_levels.hdf5
 8-8-ESN-L.hdf5				  'Fall 2017'
 8-8growth_rate.ipynb			  'Freshman Year'
 amazon.txt				   Grades
'APAM Mentor Application 2019-2020.gdoc'   IMG_1422.JPG
 Career					  'Lowe Institute'
'CHEN RICHY Photo.JPG'			   lstm_growth.hdf5
 clouderizer				  'Markets NN Pitch.gslides'
'Code Translation Project'		   Philosophy
'Colab Notebooks'			   QQtrader
'Derivatives MG Case.gdoc'		  'Quotes Thoughts.gdoc'
 Echo_State_Networks.ipynb		  'Salvatori Center'
'ESN 8-8growth_rate.ipynb'		   Schedules.gsheet
 ESN-8-to-8levels.ipynb


In [0]:
"""
  Takes a model and data, modifies the data for the model, and runs the model
"""

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras import backend as K
from numpy import concatenate
from numpy import append
from numpy import array
from numpy import nan

def sign_ae(x, y):
    sign_x = K.sign(x)
    sign_y = K.sign(y)
    delta = x - y
    return sign_x * sign_y * K.abs(delta)


def linex_loss(delta, a=-1, b=1):
    if a!= 0 and b > 0:
        loss = b * (K.exp(a * delta) - a * delta - 1)
        return loss
    else:
        raise ValueError


def linex_loss_val(y_true, y_pred):
    delta = sign_ae(y_true, y_pred)
    res = linex_loss(delta)
    return K.mean(res)


def linex_loss_ret(y_true, y_pred):
    diff_true = y_true[1:] - y_true[:-1]
    diff_pred = y_pred[1:] - y_pred[:-1]

    delta = sign_ae(diff_true, diff_pred)
    res = linex_loss(delta)
    return K.mean(res)

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
      Creates batches of data for the model
    """
    n_vars = 1 if isinstance(data, list) else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def get_data():
    data = pd.read_csv(filepath_or_buffer="https://raw.githubusercontent.com/richychn/market-modeling/master/static/data/levels.csv", index_col="date")
    data = data.apply(pd.to_numeric, errors = "coerce")
    data['spindx'].replace(0, nan, inplace=True)
    data['spindx'].fillna(method='ffill', inplace=True)
    values = data[['spindx'] + ['TCMNOM_Y2'] + ['TCMNOM_Y10'] + ['DCOILBRENTEU'] + ['GOLDPMGBD228NLBM'] + ['exalus'] + ['exjpus'] + ['exukus']].values
    values = values.astype('float32')
    return values

def data_setup(values, start):
    historic_data = array([])
    for day in values[-20:]:
        historic_data = concatenate((historic_data, day), axis=None)
    historic_data = append (historic_data, historic_data)
    look_back = 10
    time_steps = 10
    reframed = series_to_supervised(values, look_back, time_steps)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(reframed)
    pred_para = scaled[-1][8*start:8*(start+time_steps)]
    pred_para = pred_para.reshape(1,10,8)
    return pred_para    

def test_single_investment():
    values = get_data()
    K.clear_session()
    multi_model = load_model("gdrive/My Drive/8-8-ESN-L.hdf5", custom_objects={'linex_loss_ret': linex_loss_ret})

    investments = []
    earn = 1
    day = 0
    while day < len(values) / 8 - 10:
        pred_para = data_setup(values, day)
        yhat = multi_model.predict(pred_para)
        growth = yhat[0][0] - pred_para[0][-1]
 
        best_so_far = 0
        asset = -1
        for num, growth in enumerate(growth):
            if growth > best_so_far:
                best_so_far = growth
                asset = num
        if best_so_far > 0:
            earn = earn * (1 + values[day + 10][asset])
        investments.append(asset)
        day += 1
    return earn, investments
        

In [13]:
ret = test_single_investment()
print(ret)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  

ValueError: ignored